In [ ]:
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install sentence-transformers
!pip install pinecone

### Config things

In [130]:
import os

# Получаем текущий рабочий каталог
current_dir = os.getcwd()

# Строим путь на три уровня вверх
config_path = os.path.join(current_dir, '..', '..', 'llama.config')

# Нормализуем путь (убираем лишние '..' и т.д.)
config_path = os.path.normpath(config_path)


# Читаем файл
with open(config_path, 'r') as file:
    api_base = file.readline().strip()
    my_model = file.readline().strip()
    API_KEY = file.readline().strip()
    YOUR_API_KEY = file.readline().strip()
#    index_name = file.readline().strip()
index_name = 'red-llama-bertopic'

# Выводим значения
#print(f"API Base: {api_base}")
#print(f"My Model: {my_model}")
#print(f"API Key: {API_KEY}")
print(f"Pinecone API Key: {YOUR_API_KEY}")
print(f"Pinecone index name: {index_name}")


Pinecone API Key: pcsk_2MHdUF_M8e8hZxWQN2NFvQ4CWhXvsyL7EwPzNpvmy7iYRCN8hpey5u1TvqHZJ89TjJmZUx
Pinecone index name: red-llama-bertopic


### Подготовка чанков

In [23]:
from config import my_dir

# Пример использования
folder_name = 'data\\'  # Укажите имя папки, к которой хотите получить доступ
path = my_dir(folder_name)
print(path)

f:\ipynb\data


In [24]:
import pandas as pd

my_chunks = []
#my_dir = "data\\"
my_dir = my_dir('data')+"\\"


def make_chunks (file_name):
    file_path = my_dir + file_name + ".csv"
    final_df = pd.read_csv(file_path, sep=';')
    for index, row in final_df.iterrows():
        chunk = {
            "chunk_text": row['text'],
            "topic": row['topic']
            }    
        my_chunks.append(chunk)

my_file = "╨Ф╨░╤В╨░╤Б╨╡╤В_╨в╨Ъ_╨а╨д_╨╕_╨б╨Я35.txt"
make_chunks (my_file)
my_file = "rmr, World Class, Michelin (2).txt"
make_chunks (my_file)
my_file = "FAQ.txt"
make_chunks (my_file)
my_file = "metadata.txt"
make_chunks (my_file)
my_file = "╨Ф╨╖1.txt"
make_chunks (my_file)
#print(my_chunks)

# Путь к выходному файлу
output_file_path = my_dir + "output.txt"

# Открытие файла для записи
with open(output_file_path, 'w', encoding='utf-8') as file:
    # Итерация по списку чанков
    for chunk in my_chunks:
        # Форматирование строки для записи
        line = f"Text: {chunk['chunk_text']}Topic: {chunk['topic']}\n\n"
        # Запись строки в файл
        file.write(line)

print(f"Данные успешно записаны в файл: {output_file_path}")

Данные успешно записаны в файл: f:\ipynb\data\output.txt


Чанки успешно сохранены в Pinecone!


### Предобработка

In [25]:
import spacy

# Загрузка модели для русского языка
nlp = spacy.load("ru_core_news_sm")

def preprocess_text(text):
    # Удаление пунктуации и чисел
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    # Лемматизация
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return " ".join(lemmas)

### С русской предобработкой

In [27]:
import re

def extract_qa_chunks_sep(file_dir, file_name, lang_ru=True):
    file_path = file_dir + file_name
    with open(file_path, encoding='utf-8') as f:
        content = f.read()

    if (file_name=="╨Ф╨╖1.txt"):
        # Измененное регулярное выражение
#        qa_pairs = re.findall(r'\*\*(.*?)\*\*\s*(.*?)\s*\*\*', content, re.DOTALL)
        qa_pairs = re.findall(r'(.+?)(?:\n|$)\s*(.+?)(?=\n\s*\n|$)', content, re.DOTALL)

    else:
        qa_pairs = re.findall(r'(.*?)\?\s*(.+?)(?=(?:\n.*?\?|\Z))', content, re.DOTALL)


    chunks = {}
#    chunks = []
    for idx, (question, answer) in enumerate(qa_pairs):
        question = question.strip()
        answer = answer.strip()
#        print(f"Вопрос: {question}, Ответ: {answer}")  # Отладочное сообщение
#        text = f"{question}? {answer}"

        # Предположим, что lang_ru — это булева переменная, указывающая, является ли язык русским
        if lang_ru:
            # Если язык русский, применяем предобработку текста
            text = preprocess_text(f"{question}? {answer}")
        else:
            # Если язык не русский, оставляем текст без изменений
            text = f"{question}? {answer}"
        print(f"Вопрос: {question}, Ответ: {answer}")  # Отладочное сообщение

        chunks[idx] = {  # Используем индекс как ключ
        
#        chunk = {
            "id": f"chunk_{idx}",
            "text": text,
            "question": question + "?",
            "answer": answer,
            "source": file_name
        
        }
        '''
        chunk_text = f"Question: {question}\nAnswer: {answer}\n\n"
        text = f"{question}\n{answer}\n\n"
        chunks[idx] = {  # Используем индекс как ключ
            "text": text,  # Текст (вопрос и ответ)
            "chunk": chunk_text,  # Чанк (словарь с текстом и метаданными)
            "file_name": file_name
        }
        
        '''
#        chunks.append(chunk)
            # Преобразуем словарь в DataFrame
    df = pd.DataFrame.from_dict(chunks, orient='index')
    return df



### Загрузка датасета

In [28]:

combined_df = pd.DataFrame() # создание нового пустого DataFrame
my_file = "╨Ф╨╖1.txt"
combined_df = extract_qa_chunks_sep(my_dir, my_file)
#combined_df.info

my_file = "FAQ.txt"
chunks_df = pd.DataFrame() # создание нового пустого DataFrame
chunks_df = extract_qa_chunks_sep(my_dir, my_file)
combined_df = pd.concat([combined_df, chunks_df], ignore_index=True)

# pd.DataFrame() # создание нового пустого DataFrame
my_file = "metadata.txt" # АНГЛ.
chunks_df = pd.DataFrame() # создание нового пустого DataFrame
chunks_df = extract_qa_chunks_sep(my_dir, my_file, False)
combined_df = pd.concat([combined_df, chunks_df], ignore_index=True)


my_file = "rmr, World Class, Michelin (2).txt"
chunks_df = pd.DataFrame() # создание нового пустого DataFrame
chunks_df = extract_qa_chunks_sep(my_dir, my_file)
combined_df = pd.concat([combined_df, chunks_df], ignore_index=True)

my_file = "╨Ф╨░╤В╨░╤Б╨╡╤В_╨в╨Ъ_╨а╨д_╨╕_╨б╨Я35.txt"
chunks_df = pd.DataFrame() # создание нового пустого DataFrame
chunks_df = extract_qa_chunks_sep(my_dir, my_file)
combined_df = pd.concat([combined_df, chunks_df], ignore_index=True)
print(combined_df.shape)
combined_df.columns
combined_df.info


Вопрос: **Что такое обещанный платёж и как его подключить?**, Ответ: Обещанный платёж поможет сохранить доступ к услугам при отрицательном или нулевом балансе на 5 суток. Подключить услугу могут клиенты, которые пользуются только интернетом, а также двумя и более услугами, например, интернетом+ТВ. Стоимость обещанного платежа — 39 ₽/день. Оплата списывается за дни, когда услуга будет действовать. Обещанный платёж отключается на следующий день после поступления на договор денежных средств. Для отключения обещанного платежа необходимо внести сумму задолженности + стоимость обещанного платежа за все дни, в которые вы пользовались услугой.
Вопрос: , Ответ: **Как проверить баланс и историю в мобильном приложении на iOS?**
	Откройте мобильное приложение «Мой Дом.ру». На экране Главная вы увидите сумму рекомендуемого платежа и текущий баланс. Нажмите на кнопку История платежей и начеслений. В открывшемся разделе будет представлена история начислений и списаний, для просмотра предыдущих период

<bound method DataFrame.info of            id                                               text  \
0     chunk_0  что такой обещать платёж и как его подключить ...   
1     chunk_1    как проверить баланс и история в мобильный п...   
2     chunk_2    какой номер вводить при оплата \n\t При опла...   
3     chunk_3    не активировать тариф что делать \n\t в перв...   
4     chunk_4    почему я должный платить если уже не жить по...   
..        ...                                                ...   
375  chunk_95    какой мера против скольжение применяться шер...   
376  chunk_96    как рассчитываться шумозащита на основа уров...   
377  chunk_97    какой требование к пешеходный переход на мос...   
378  chunk_98    какой ограничение по скорость движение на мо...   
379  chunk_99    какой перспектива развитие мостостроение ука...   

                                              question  \
0    **Что такое обещанный платёж и как его подключ...   
1                                  

In [29]:
chunks_df = combined_df
print(chunks_df.shape)
chunks_df.info

(380, 5)


<bound method DataFrame.info of            id                                               text  \
0     chunk_0  что такой обещать платёж и как его подключить ...   
1     chunk_1    как проверить баланс и история в мобильный п...   
2     chunk_2    какой номер вводить при оплата \n\t При опла...   
3     chunk_3    не активировать тариф что делать \n\t в перв...   
4     chunk_4    почему я должный платить если уже не жить по...   
..        ...                                                ...   
375  chunk_95    какой мера против скольжение применяться шер...   
376  chunk_96    как рассчитываться шумозащита на основа уров...   
377  chunk_97    какой требование к пешеходный переход на мос...   
378  chunk_98    какой ограничение по скорость движение на мо...   
379  chunk_99    какой перспектива развитие мостостроение ука...   

                                              question  \
0    **Что такое обещанный платёж и как его подключ...   
1                                  

In [30]:
# Мультиязычный датасет
docs = chunks_df['text'].dropna().loc[chunks_df['text'].str.strip() != '']

# Преобразование результата в список
docs_list = docs.tolist()
print(docs_list)

['что такой обещать платёж и как его подключить обещать платёж помочь сохранить доступ к услуга при отрицательный или нулевой баланс на   суток подключить услуга мочь клиент которые пользоваться только интернет а также два и более услуга например интернетомтв стоимость обещанного платёж    день оплата списываться за день когда услуга быть действовать обещать платёж отключаться на следующий день после поступление на договор денежный средство Для отключение обещанного платёж необходимый внести сумма задолженность   стоимость обещать платёж за весь день в которые вы пользоваться услуга', '  как проверить баланс и история в мобильный приложение на ios \n\t откройте мобильный приложение мой домру на экран главный вы увидеть сумма рекомендуемого платёж и текущий баланс нажмите на кнопка история платёж и начеслений в открыться раздел быть представить история начисление и списание для просмотр предыдущий период пролистать страница вниз Для просмотр история операция выбрать кнопка в правый верх

### Мультиязычная модель. Обучена!

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired

# Используем модель, поддерживающую русский язык
#sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
sentence_model = SentenceTransformer('distiluse-base-multilingual-cased')

# Мои настройки
my_nr_topics = 14 # Количество тем, которые вы хотите извлечь. Можете настроить это значение, чтобы контролировать количество тем.
my_nr_repr_docs=7 # Количество документов, используемых для представления каждой темы
my_min_topic_size=10  # Минимальный размер темы, сколько документов должно быть в теме, чтобы она считалась значимой
му_top_n_words=10 # Количество слов, которые вы хотите получить для каждой темы

# Настройка модели представления
representation_model = KeyBERTInspired(nr_repr_docs=my_nr_repr_docs, top_n_words=му_top_n_words)

# Создание модели BERTopic с настройками
topic_model = BERTopic(
    embedding_model=sentence_model,
    representation_model=representation_model,
    nr_topics=my_nr_topics,
    min_topic_size=my_min_topic_size,  # Минимальный размер темы
    calculate_probabilities=True,  # Вычисление вероятностей
    verbose=True  # Вывод дополнительной информации
)

# Обучение модели
#topics, probs = topic_model.fit_transform(docs)

# Сохранение модели
#topic_model.save("red_bertopic_model")

# Загрузка модели
from bertopic import BERTopic
loaded_model = BERTopic.load("red_bertopic_model")

# Добавление тем и вероятностей в DataFrame
#chunks_df['topic'] = topics
#chunks_df['probs'] = probs


# Вывод информации
print(chunks_df.columns)
print(chunks_df.info())

2025-03-29 02:30:51,505 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 12/12 [00:08<00:00,  1.49it/s]
2025-03-29 02:30:59,568 - BERTopic - Embedding - Completed ✓
2025-03-29 02:30:59,569 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-03-29 02:30:59,778 - BERTopic - Dimensionality - Completed ✓
2025-03-29 02:30:59,779 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-03-29 02:30:59,805 - BERTopic - Cluster - Completed ✓
2025-03-29 02:30:59,806 - BERTopic - Representation - Extracting topics from clusters using representation models.
2025-03-29 02:31:03,608 - BERTopic - Representation - Completed ✓
2025-03-29 02:31:03,609 - BERTopic - Topic reduction - Reducing number of topics
2025-03-29 02:31:03,610 - BERTopic - Topic reduction - Reduced number of topics from 10 to 10
2025-03-29 02:31:03,640 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the en

Index(['id', 'text', 'question', 'answer', 'source', 'topic'], dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        380 non-null    object
 1   text      380 non-null    object
 2   question  380 non-null    object
 3   answer    380 non-null    object
 4   source    380 non-null    object
 5   topic     380 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 17.9+ KB
None


In [159]:
# Получаем информацию по темам
topic_model = loaded_model
print(f"Настройки. Количество тем: {my_nr_topics}, Количество представи. документов: {my_nr_repr_docs}, Мин. размер: {my_min_topic_size}, Количество топ-слов: {му_top_n_words}")
topic_info = topic_model.get_topic_info()
print(topic_info)


Настройки. Количество тем: 14, Количество представи. документов: 7, Мин. размер: 10, Количество топ-слов: 10
   Topic  Count                                        Name  \
0     -1     21  -1_бакалавриат_бакалавр_диплом_аспирантура   
1      0     59     0_французский_бульдог_собака_популярный   
2      1     22     1_red_mad_robot_red_цифровой_разработка   
3      2     54            2_telegram_messages_message_chat   
4      3     62                 3_кожа_кислота_витамин_вода   
5      4     48         4_мост_бетон_строительство_строение   
6      5     19      5_звезда_ресторан_трёхзвёздочный_кухня   
7      6     50   6_работодатель_трудовой_работник_зарплата   
8      7     14              7_class_тренировка_клуб_тренер   
9      8     31       8_платёж_фискальный_оплачивать_оплата   

                                      Representation  \
0  [бакалавриат, бакалавр, диплом, аспирантура, о...   
1  [французский, бульдог, собака, популярный, щен...   
2  [red_mad_robot, red, цифро

In [83]:
# Получаем ключевые слова по темам
print(f"Количество тем заказано: {my_nr_topics}, Количество представи. документов: {my_nr_repr_docs}, Мин. размер: {my_min_topic_size}, Количество топ-слов: {му_top_n_words}")

for id in range(-1, len(topic_info)-1):
    topic_words = topic_model.get_topic(id)
    print(f"Ключевые слова по теме {id}: {topic_words}")


Количество тем заказано: 14, Количество представи. документов: 7, Мин. размер: 10, Количество топ-слов: 10
Ключевые слова по теме -1: [('бакалавриат', np.float32(0.31518373)), ('бакалавр', np.float32(0.27087903)), ('диплом', np.float32(0.2588203)), ('аспирантура', np.float32(0.23891361)), ('образование', np.float32(0.23076478)), ('студент', np.float32(0.19909294)), ('учебный', np.float32(0.18908706)), ('специальность', np.float32(0.18783751)), ('обучение', np.float32(0.18729722)), ('специалитета', np.float32(0.10582714))]
Ключевые слова по теме 0: [('французский', np.float32(0.47135442)), ('бульдог', np.float32(0.15758193)), ('собака', np.float32(0.14411372)), ('популярный', np.float32(0.1424608)), ('щенок', np.float32(0.12345127)), ('порода', np.float32(0.09492709)), ('животное', np.float32(0.0780914)), ('умный', np.float32(0.07168501)), ('дружелюбный', np.float32(0.05522666)), ('компактный', np.float32(0.04894419))]
Ключевые слова по теме 1: [('red_mad_robot', np.float32(0.47562537))

### Темы в старой модели и индексе:
-    -1 Правила оказания услуг
-    6 Трудовой кодекс
-    2 Строительство
-    1 Уход за кожей
-    0 Французский бульдог
-    4 Telegram
-    5 rmr (red_mad_robot)
-    3 Michelin
-    7 World Class

### В новой модели изменились темы
#### **Правила оказания услуг** разделились на **Социальное обеспечение** и **Оплату услуг**

- -1: Социальное обеспечение
- 0: Французский бульдог
- 1: rmr (red_mad_robot)
- 2: Telegram
- 3: Уход за кожей
- 4: Мостостроительство
- 5: Michelin
- 6: Трудовой кодекс
- 7: World Class
- 8: Оплата услуг

In [123]:
topics = {
    -1: "Социальное обеспечение",
    0: "Французский бульдог",
    1: "rmr (red_mad_robot)",
    2: "Telegram",
    3: "Уход за кожей",
    4: "Мостостроительство",
    5: "Michelin",
    6: "Трудовой кодекс",
    7: "World Class",
    8: "Оплата услуг"
}

# Чтение словаря в цикле
for key, value in topics.items():
    print(f"{key}: {value}")

-1: Социальное обеспечение
0: Французский бульдог
1: rmr (red_mad_robot)
2: Telegram
3: Уход за кожей
4: Мостостроительство
5: Michelin
6: Трудовой кодекс
7: World Class
8: Оплата услуг


In [124]:
chunks_df.info

<bound method DataFrame.info of            id                                               text  \
0     chunk_0  что такой обещать платёж и как его подключить ...   
1     chunk_1    как проверить баланс и история в мобильный п...   
2     chunk_2    какой номер вводить при оплата \n\t При опла...   
3     chunk_3    не активировать тариф что делать \n\t в перв...   
4     chunk_4    почему я должный платить если уже не жить по...   
..        ...                                                ...   
375  chunk_95    какой мера против скольжение применяться шер...   
376  chunk_96    как рассчитываться шумозащита на основа уров...   
377  chunk_97    какой требование к пешеходный переход на мос...   
378  chunk_98    какой ограничение по скорость движение на мо...   
379  chunk_99    какой перспектива развитие мостостроение ука...   

                                              question  \
0    **Что такое обещанный платёж и как его подключ...   
1                                  

### Поиск по новой новой обученной модели red_bertopic_model

In [166]:
from bertopic import BERTopic
import pandas as pd


# Ваши новые документы
new_docs = [
    "Правила оплаты труда наемного работник",
    "Где лучше заниматься спортом и фитнесом?",
    "Сколько учиться на бакалавриате?"
]

# Получение тем и вероятностей для новых документов
new_topics, new_probs = topic_model.transform(new_docs)

# Создание DataFrame для новых документов
new_chunks_df = pd.DataFrame({
    'document': new_docs,
    'topic': new_topics
})

# Вывод DataFrame с темами
print(new_chunks_df[['document', 'topic']])


Batches: 100%|██████████| 1/1 [00:00<00:00, 28.97it/s]
2025-03-29 04:55:21,394 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-03-29 04:55:21,401 - BERTopic - Dimensionality - Completed ✓
2025-03-29 04:55:21,402 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-03-29 04:55:21,402 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2025-03-29 04:55:21,404 - BERTopic - Probabilities - Completed ✓
2025-03-29 04:55:21,404 - BERTopic - Cluster - Completed ✓


                                   document  topic
0    Правила оплаты труда наемного работник      6
1  Где лучше заниматься спортом и фитнесом?      7
2          Сколько учиться на бакалавриате?     -1


### Создаем чанки для новой модели

In [125]:
import pandas as pd

final_df = chunks_df
my_chunks = []
for index, row in final_df.iterrows():
    # Чтение словаря в цикле
    for key, value in topics.items():
        if (key == row['topic']):
            topic_name = value
    chunk = {
        "chunk_text": row.get('text', 'Нет текста'),
        "question": row.get('question', 'Нет вопроса'),  # Убедитесь, что этот ключ существует
        "answer": row.get('answer', 'Нет ответа'),
        "source": row.get('source', 'Нет источника'),
        "topic": row.get('topic', 'Нет темы'),
        "topic_name": topic_name
}
    my_chunks.append(chunk)

print(my_chunks)

[{'chunk_text': 'что такой обещать платёж и как его подключить обещать платёж помочь сохранить доступ к услуга при отрицательный или нулевой баланс на   суток подключить услуга мочь клиент которые пользоваться только интернет а также два и более услуга например интернетомтв стоимость обещанного платёж    день оплата списываться за день когда услуга быть действовать обещать платёж отключаться на следующий день после поступление на договор денежный средство Для отключение обещанного платёж необходимый внести сумма задолженность   стоимость обещать платёж за весь день в которые вы пользоваться услуга', 'question': '**Что такое обещанный платёж и как его подключить?**?', 'answer': 'Обещанный платёж поможет сохранить доступ к услугам при отрицательном или нулевом балансе на 5 суток. Подключить услугу могут клиенты, которые пользуются только интернетом, а также двумя и более услугами, например, интернетом+ТВ. Стоимость обещанного платежа — 39 ₽/день. Оплата списывается за дни, когда услуга б

In [126]:
print(final_df.columns)

Index(['id', 'text', 'question', 'answer', 'source', 'topic'], dtype='object')


### Проверка формирования чанков. Запись

In [127]:

# Путь к выходному файлу
output_file_path = "data\output_new.txt"

# Открытие файла для записи
with open(output_file_path, 'w', encoding='utf-8') as file:
    # Итерация по списку чанков
    for chunk in my_chunks:
        # Форматирование строки для записи
        line = f"Text: {chunk['chunk_text']}, question: {chunk['question']}, answer: {chunk['answer']}, source: {chunk['source']}, topic: {chunk['topic']}, topic_name: {chunk['topic_name']}\n\n"

        # Запись строки в файл
        file.write(line)

print(f"Данные успешно записаны в файл: {output_file_path}")



Данные успешно записаны в файл: data\output_new.txt


## Pinecone Things
### Сохранение чанков в Pinecone

In [135]:

# from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

# Загрузка модели
model = SentenceTransformer('all-MiniLM-L6-v2')

# Векторизация чанков
chunk_embeddings = model.encode(my_chunks)

# Инициализация Pinecone
pc = Pinecone(api_key=YOUR_API_KEY)  # Замените на ваш API-ключ

# Создание индекса
index_name = "red-llama-bertopic"
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Размерность эмбеддингов
        metric="cosine",  # Метрика для поиска
        spec=ServerlessSpec(
            cloud="aws",  # Облачный провайдер
            region="us-east-1"  # Регион
        )
    )

# Подключение к индексу
index = pc.Index(index_name)

# Сохранение чанков в Pinecone
for i, (chunk, embedding) in enumerate(zip(my_chunks, chunk_embeddings)):
    index.upsert([(f"chunk_{i}", embedding, chunk)])

print("Чанки успешно сохранены в Pinecone!")


Чанки успешно сохранены в Pinecone!


### Нахождение темы по локальной обученной модели

In [175]:
# Ваши новые документы
new_docs = [
    "Правила оплаты труда наемного работник",
    "Сколько учиться на бакалавриате?",
]

# Получение тем и вероятностей для новых документов
new_topics, new_probs = topic_model.transform(new_docs)

# Создание DataFrame для новых документов
new_chunks_df = pd.DataFrame({
    'document': new_docs,
    'topic': new_topics
})

# Вывод DataFrame с темами
print(new_chunks_df[['document', 'topic']])


Batches: 100%|██████████| 1/1 [00:00<00:00, 38.45it/s]
2025-03-29 05:13:08,058 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-03-29 05:13:08,064 - BERTopic - Dimensionality - Completed ✓
2025-03-29 05:13:08,065 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-03-29 05:13:08,065 - BERTopic - Probabilities - Start calculation of probabilities with HDBSCAN
2025-03-29 05:13:08,067 - BERTopic - Probabilities - Completed ✓
2025-03-29 05:13:08,067 - BERTopic - Cluster - Completed ✓


                                 document  topic
0  Правила оплаты труда наемного работник      6
1        Сколько учиться на бакалавриате?     -1


### Отличный результат

### Поиск в Pinecone c фильтрацией

In [ ]:
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

# Инициализация Pinecone
#pc = Pinecone(api_key=YOUR_API_KEY)  # Замените на ваш API-ключ

# Подключение к индексу
index_name = "red-llama-bertopic"
index = pc.Index(index_name)

# Загрузка модели
model = SentenceTransformer('all-MiniLM-L6-v2')

'''
# Поиск в индексе с фильтрацией
filter_condition = {
    "topic_name": {
        "$eq": "Социальное обеспечение" 
    }
}
'''
# Поиск в индексе с фильтрацией
filter_condition = {
    "topic": {
        "$eq": -1 
    }
}

# Векторизация запроса
query_text = "Сколько учиться на бакалавриате?"
query_embedding = model.encode(query_text)
# Поиск в индексе
results = index.query(
    vector=query_embedding.tolist(),  # Вектор запроса
    top_k=5,  # Количество возвращаемых результатов
    include_metadata=True,  # Включить метаданные в результаты
    filter=filter_condition  # Условия фильтрации

)
# Вывод результатов
for match in results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}, answer: {match['metadata']['answer']}\n Тема: {match['metadata']['topic_name']}")


ID: chunk_7, Score: 0.595531106, answer: **Как оплатить услуги бонусами от Дом.ру?**
	Участвуйте в программе Дом.ру Бонус. Так на вашем счёте будут копиться бонусы, которыми можно оплатить до 100% стоимости абонентской платы.​ 100 бонусов = 20 ₽. А ещё получайте скидки на услуги наших партнёров.
 Тема: Социальное обеспечение
ID: chunk_33, Score: 0.585178196, answer: **Какие Минусы специалитета?**
	К минусам специалитета можно отнести более долгий срок обучения. Кроме того, возможность получить специальность есть не во всех вузах.
 Тема: Социальное обеспечение
ID: chunk_39, Score: 0.526665151, answer: **Что будет с дипломами, полученным до введения базового высшего образования?**
	Дипломы останутся полноценными и будут иметь прежнюю силу. Аналогичной была история с дипломами, которые были получены в СССР и в России до появления бакалавриата и специалитета.
 Тема: Социальное обеспечение
ID: chunk_36, Score: 0.515411735, answer: **Отличия бакалавриата от специалитета**
	Ключевые различия 

In [173]:
# Поиск в индексе с фильтрацией
filter_condition = {
    "topic": {
        "$eq": 6 
    }
}

# Векторизация запроса
query_text = "Правила оплаты труда наемного работник"
query_embedding = model.encode(query_text)
# Поиск в индексе
results = index.query(
    vector=query_embedding.tolist(),  # Вектор запроса
    top_k=5,  # Количество возвращаемых результатов
    include_metadata=True,  # Включить метаданные в результаты
    filter=filter_condition  # Условия фильтрации

)
# Вывод результатов
for match in results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}, answer: {match['metadata']['answer']}\n Тема: {match['metadata']['topic_name']}")


ID: chunk_324, Score: 0.776943266, answer: Предоставляются пособие по беременности и родам, единовременное пособие и ежемесячные выплаты по уходу за ребенком.
 Тема: Трудовой кодекс
ID: chunk_329, Score: 0.738587141, answer: Работник имеет право на уведомление об изменениях за два месяца и на отказ от работы в случае несогласия.
 Тема: Трудовой кодекс
ID: chunk_295, Score: 0.735562146, answer: Как правильно написать заявление?
 Тема: Трудовой кодекс
ID: chunk_316, Score: 0.731703639, answer: Работнику возмещаются расходы на проезд, проживание, суточные и сохраняется средняя зарплата.
 Тема: Трудовой кодекс
ID: chunk_303, Score: 0.726858556, answer: Работа в выходные и праздничные дни допускается только с согласия работника и оплачивается в двойном размере.
 Тема: Трудовой кодекс
